In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
warnings.filterwarnings("ignore", category=ConvergenceWarning)
import os
import warnings
warnings.filterwarnings('ignore')


### Preprocessing

In [ ]:
df = pd.read_csv('../build_dataset/output_csv/bloomberg_data.csv')
df.head(3)

In [63]:
def convert_trade_date(df):
    """
    Convert the 'Trade Date (US)' column to separate columns for 
    month, day, and year, and drop the original 'Trade Date (US)' column.
    """
    df['Trade Date (US)'] = pd.to_datetime(df['Trade Date (US)'])
    df['Trade Month'] = df['Trade Date (US)'].dt.month
    df['Trade Day'] = df['Trade Date (US)'].dt.day
    df['Trade Year'] = df['Trade Date (US)'].dt.year
    df.drop(columns=['Trade Date (US)'], inplace=True)
    return df

df_convert_trade_date = convert_trade_date(df)

In [64]:
def drop_columns(columns,df):
    """
    Drop the specified columns from the DataFrame.
    """
    df = df.drop(columns=columns)
    return df

df_drop_columns = drop_columns(['Issuer Ticker',
                                'Issuer Name',
                                'Filing Term Price Range',
                                'cusip',
                                'Priced Range'],
                                df_convert_trade_date)

In [65]:
def encoding(df):
    """
    Split the DataFrame into development and testing sets. 
    
    Fill missing values with the mode for categorical 
    columns and the mean for numerical columns.

    Encode columns in the DataFrame: 

      - Categorical columns are encoded using OneHotEncoder.
      - Numerical columns are scaled using StandardScaler.
      - Ordinal columns are encoded using OrdinalEncoder.

    Create df_dev and df_test using the transformed features.
    """
    X = df.drop(columns=['Offer To 1st Close'])
    y = df['Offer To 1st Close']

    X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    if y_dev.isnull().any() or y_test.isnull().any():
        mean = y_dev.mean()
        y_dev = y_dev.fillna(mean)
        y_test = y_test.fillna(mean)

    for col in X_dev.columns:
        if X_dev[col].dtype == 'object':
            mode = X_dev[col].mode()[0]
            X_dev[col] = X_dev[col].fillna(mode)
            X_test[col] = X_test[col].fillna(mode)
        else:
            mean = X_dev[col].mean()
            X_dev[col] = X_dev[col].fillna(mean)
            X_test[col] = X_test[col].fillna(mean)

    oe_columns = ['Trade Month', 'Trade Day', 'Trade Year'] 
    ohe_columns = ['Industry Sector', 'Industry Group', 'Industry Subgroup']
    ss_columns = [col for col in X.select_dtypes(exclude=['object']).columns if col not in oe_columns]


    oe = OrdinalEncoder()
    ohe = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
    ss = StandardScaler()

    preprocess = ColumnTransformer(transformers=[
        ('ohe', ohe, ohe_columns),
        ('ss', ss, ss_columns),
        ('oe', oe, oe_columns)
    ], remainder='passthrough')

    transformed_dev = preprocess.fit_transform(X_dev)
    transformed_test = preprocess.transform(X_test)

    features = preprocess.get_feature_names_out()

    df_dev = pd.DataFrame(transformed_dev, columns=features)
    df_test = pd.DataFrame(transformed_test, columns=features)

    df_dev['Offer To 1st Close'] = y_dev.reset_index(drop=True)
    df_test['Offer To 1st Close'] = y_test.reset_index(drop=True)

    return df_dev, df_test


df_dev_encoding, df_test_encoding = encoding(df_drop_columns)

In [ ]:
def show_correlation(df_dev, df_test):
    """
    Check correlation in the X_dev column and drop highly correlated features.
    """
    X_dev = df_dev.drop(columns=['Offer To 1st Close'])
    y_dev = df_dev['Offer To 1st Close']

    X_test = df_test.drop(columns=['Offer To 1st Close'])
    y_test = df_test['Offer To 1st Close']

    corr_matrix = X_dev.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] >= 0.9)]

    X_reduced = X_dev.drop(columns=to_drop)
    X_reduced_test = X_test.drop(columns=to_drop)

    df_dev = pd.concat([X_reduced, y_dev.reset_index(drop=True)], axis=1)
    df_test = pd.concat([X_reduced_test, y_test.reset_index(drop=True)], axis=1)

    print(f"Removed {len(to_drop)} correlated features.")

    df_test.to_csv('./output_csv/Final_Output_Reg_test.csv', index=False)

    return df_dev, df_test

df_dev_correlation, df_test_correlation = show_correlation(df_dev_encoding, df_test_encoding)

In [67]:
def remove_outlier(df):
    """
    Remove rows that correspond to outliers in the target variable
    """
    label = df['Offer To 1st Close']
    Q1 = label.quantile(0.25)
    Q3 = label.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df_filtered = df[(label >= lower_bound) & (label <= upper_bound)]
    print(f"Removed outliers.")
    print(f"Original count: {len(df)}, New count: {len(df_filtered)}")
    return df_filtered


In [ ]:
def checkoutliers(df, outfile):
    """
    Print distribution of the target variable and check for outliers.
    """
    label = df['Offer To 1st Close']

    Q1 = label.quantile(0.25)
    Q3 = label.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    print('Minimum value: ', label.min())
    print('Q1: ', Q1)
    print('Median value: ', label.median())
    print('Q3: ', Q3)
    print('Maximum value: ', label.max())
    print('Mean value: ', label.mean())
    print('Standard deviation: ', label.std())

    outliers = (label < lower_bound).any() or (label > upper_bound).any()

    if outliers:
        print('\nThere are outliers in the data.')
        df_filtered = remove_outlier(df)
        df_filtered.to_csv(outfile, index=False)
        return df_filtered
    else:
        print('\nThere are no outliers in the data.')
        return df

df_dev_filtered = checkoutliers(df_dev_correlation, './output_csv/Final_Output_Reg_dev.csv')

In [ ]:
def display_distribution(df_dev_filtered, df_dev_correlation):
    df_no_outliers = df_dev_filtered.copy()
    df_with_outliers = df_dev_correlation.copy()

    fig, axs = plt.subplots(ncols=2, figsize=(15, 10))

    sns.boxplot(data=df_with_outliers, y='Offer To 1st Close', color='lightblue', ax=axs[0])
    axs[0].set_title('With Outliers')
    axs[0].set_xlabel('Offer to 1st Close')

    sns.boxplot(data=df_no_outliers, y='Offer To 1st Close', color='lightblue', ax=axs[1])
    axs[1].set_title('Removed Outliers')
    axs[1].set_xlabel('Offer to 1st Close')

    fig.suptitle('Regression Task: Distribution of Offer to 1st Close', fontsize=14)
    plt.tight_layout()
    plt.show()

display_distribution(df_dev_filtered, df_dev_correlation)

### Model Training

In [70]:
def split_data(df, target_col='Offer To 1st Close'):
    """Preprocess the data with feature engineering and scaling"""
    X = df.drop(columns=target_col)
    y = df[target_col]
    return X, y

In [71]:
def get_enhanced_model_params():
    """Return enhanced model parameters with wider search space"""
    return {
        "Linear Regression": {
            "model": LinearRegression(n_jobs=-1),
            "params": {}
        },
        "Ridge Regression": {
            "model": Ridge(random_state=42, max_iter=10000),
            "params": {
                "alpha": [0.001, 0.01, 0.1, 1, 10, 100, 1000]
            }
        },
        "Lasso Regression": {
            "model": Lasso(random_state=42, max_iter=10000),
            "params": {
                "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
            }
        },
        "ElasticNet Regression": {
            "model": ElasticNet(random_state=42, max_iter=10000),
            "params": {
                "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10],
                "l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9]
            }
        },
        "Support Vector Regressor": {
            "model": SVR(),
            "params": {
                "kernel": ["rbf", "poly"],
                "C": [0.1, 1, 10],
                "gamma": ["scale", "auto"]
            }
        },
        "LightGBM": {
            "model": LGBMRegressor(random_state=42, n_jobs=-1),
            "params": {
                "n_estimators": [100, 200, 500],
                "learning_rate": [0.001, 0.01, 0.1],
                "max_depth": [3, 5, 7, -1],
                "num_leaves": [15, 31, 63],
                "subsample": [0.8, 0.9, 1.0],
                "verbosity": [-1]  
            }
        }
    }

In [72]:
def evaluate_models(model_params, X_train, y_train, X_val, y_val):
    """Evaluate models with enhanced metrics and display progress"""
    results = []

    total_models = len(model_params)
    print(f"Evaluating {total_models} models...\n")

    for i, (name, mp) in enumerate(model_params.items(), 1):
        print(f"[{i}/{total_models}] Training and evaluating: {name}...", end=" ")

        try:
            if mp["params"]:
                grid = GridSearchCV(
                    mp["model"], 
                    mp["params"], 
                    cv=5, 
                    scoring='r2', 
                    n_jobs=-1,
                    verbose=0
                )
                grid.fit(X_train, y_train)
                best_model = grid.best_estimator_
                print("✓ (GridSearch complete)")
            else:
                best_model = mp["model"]
                best_model.fit(X_train, y_train)
                print("✓ (No tuning needed)")
            
            # Cross-validation scores
            cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='r2')
            
            y_pred = best_model.predict(X_val)
            r2 = r2_score(y_val, y_pred)
            mae = mean_absolute_error(y_val, y_pred)
            mse = mean_squared_error(y_val, y_pred)
            rmse = np.sqrt(mse)
            
            results.append({
                "Model": name,
                "R² Score": r2,
                "MAE": mae,
                "MSE": mse,
                "RMSE": rmse,
                "CV R² Mean": cv_scores.mean(),
                "CV R² Std": cv_scores.std()
            })
            
            # Plot learning curves
            train_sizes, train_scores, val_scores = learning_curve(
                best_model, X_train, y_train, cv=5, scoring='r2',
                train_sizes=np.linspace(0.1, 1.0, 10)
            )
            
            plt.figure(figsize=(10, 6))
            plt.plot(train_sizes, train_scores.mean(axis=1), label='Training score')
            plt.plot(train_sizes, val_scores.mean(axis=1), label='Cross-validation score')
            plt.fill_between(train_sizes, 
                           train_scores.mean(axis=1) - train_scores.std(axis=1),
                           train_scores.mean(axis=1) + train_scores.std(axis=1),
                           alpha=0.1)
            plt.fill_between(train_sizes,
                           val_scores.mean(axis=1) - val_scores.std(axis=1),
                           val_scores.mean(axis=1) + val_scores.std(axis=1),
                           alpha=0.1)
            plt.title(f'Learning Curves - {name}')
            plt.xlabel('Training examples')
            plt.ylabel('R² Score')
            plt.legend()
            plt.grid(True)
            plt.show()
            
            # Plot feature importance for tree-based models, if applicable
            if hasattr(best_model, 'feature_importances_'):
                importances = best_model.feature_importances_
                indices = np.argsort(importances)[::-1]
                plt.figure(figsize=(10, 6))
                plt.title(f'Feature Importances - {name}')
                plt.bar(range(X_train.shape[1]), importances[indices])
                plt.xticks(range(X_train.shape[1]), X_train.columns[indices], rotation=90)
                plt.tight_layout()
                plt.show()
                
        except Exception as e:
            print(f"✗ Error with {name}: {str(e)}")
            continue

In [73]:
def clean_column_names(df):
    df.columns = df.columns.str.replace(r"[^\w\d_]+", "_", regex=True)
    return df

In [74]:
def main():
    try:
        df_dev = pd.read_csv('./output_csv/Final_Output_Reg_dev.csv')
        X, y = split_data(df_dev)
        X = clean_column_names(X)
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        
        model_params = get_enhanced_model_params()
        results_df, top5_models_df = evaluate_models(model_params, X_train, y_train, X_val, y_val)
        
        print("\nTop 5 Models:")
        print(top5_models_df)
        
        results_df.to_csv('./output_csv/regression_results.csv', index=False)
        top5_models_df.to_csv('./output_csv/top5_regression_models.csv', index=False)
        
    except Exception as e:
        print(f"Error in main execution: {str(e)}")
        raise

In [ ]:
main()